In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 27146641
paper_name = 'johnson_wu_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [62]:
original_data = pd.read_excel('raw_data/c6mt00039h1.xlsx', sheet_name='Sheet1', skiprows=1, 
                              names=['orf','h2o_t0','h2o_t16','chr5_t0','chr5_t16','chr1_t0','chr1_t16'])

In [63]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4874 x 7


In [64]:
original_data.head()

,orf,h2o_t0,h2o_t16,chr5_t0,chr5_t16,chr1_t0,chr1_t16
0,BY4743 Average (n=128),0.041889,0.900905,0.041722,0.799802,0.041778,0.511198
1,YAL002W,0.039000,0.692000,0.043000,0.110000,0.043000,0.097000
2,YAL004W,0.041000,0.960000,0.047000,0.858000,0.045000,0.480000
3,YAL005C,0.040000,0.921000,0.049000,0.866000,0.042000,0.330000
4,YAL007C,0.042000,1.047000,0.046000,0.831000,0.045000,0.456000


In [65]:
original_data['orf'] = original_data['orf'].astype(str)

In [66]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [67]:
# If possible, fix typos, omissions, etc.
original_data.loc[original_data['orf'].str.contains('BY4743AVERAGEN128'),'orf'] = 'WT'

In [68]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [69]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])

In [70]:
print(original_data.loc[~t,])

            orf    h2o_t0   h2o_t16   chr5_t0  chr5_t16   chr1_t0  chr1_t16
index_input                                                                
0            WT  0.041889  0.900905  0.041722  0.799802  0.041778  0.511198


In [71]:
# Normalize by t16 by t0, treated vs untreated and mut vs wt
original_data['h2o_ratio'] = original_data['h2o_t16'] / original_data['h2o_t0']
original_data['chr5_ratio'] = original_data['chr5_t16'] / original_data['chr5_t0']
original_data['chr1_ratio'] = original_data['chr1_t16'] / original_data['chr1_t0']

In [72]:
original_data['h2o_ratio_wt'] = original_data['h2o_ratio'] / original_data.loc[original_data['orf']=='WT','h2o_ratio'].values
original_data['chr5_ratio_wt'] = original_data['chr5_ratio'] / original_data.loc[original_data['orf']=='WT','chr5_ratio'].values
original_data['chr1_ratio_wt'] = original_data['chr1_ratio'] / original_data.loc[original_data['orf']=='WT','chr1_ratio'].values

In [74]:
original_data['chr5_ratio_wt_unt'] = original_data['chr5_ratio_wt'] / original_data['h2o_ratio_wt']
original_data['chr1_ratio_wt_unt'] = original_data['chr1_ratio_wt'] / original_data['h2o_ratio_wt']

In [75]:
original_data.head()

,orf,h2o_t0,h2o_t16,chr5_t0,chr5_t16,chr1_t0,chr1_t16,h2o_ratio,chr5_ratio,chr1_ratio,h2o_ratio_wt,chr5_ratio_wt,chr1_ratio_wt,chr5_ratio_wt_unt,chr1_ratio_wt_unt
index_input,,,,,,,,,,,,,,,
0,WT,0.041889,0.900905,0.041722,0.799802,0.041778,0.511198,21.507010,19.169679,12.236132,1.000000,1.000000,1.000000,1.000000,1.000000
1,YAL002W,0.039000,0.692000,0.043000,0.110000,0.043000,0.097000,17.743590,2.558140,2.255814,0.825014,0.133447,0.184357,0.161751,0.223459
2,YAL004W,0.041000,0.960000,0.047000,0.858000,0.045000,0.480000,23.414634,18.255319,10.666667,1.088698,0.952302,0.871735,0.874716,0.800714
3,YAL005C,0.040000,0.921000,0.049000,0.866000,0.042000,0.330000,23.025000,17.673469,7.857143,1.070581,0.921949,0.642126,0.861167,0.599792
4,YAL007C,0.042000,1.047000,0.046000,0.831000,0.045000,0.456000,24.928571,18.065217,10.133333,1.159091,0.942385,0.828148,0.813038,0.714481


In [79]:
# If the same strain is present more than once, average its values
data = original_data.groupby('orf')[['chr5_ratio_wt_unt','chr1_ratio_wt_unt']].mean()

In [82]:
data.drop(index='WT', inplace=True)

In [83]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4841 x 2


# Prepare the final dataset

In [87]:
dataset_ids = [16447, 16446]

In [88]:
datasets = datasets.reindex(index=dataset_ids)

In [89]:
data.columns = datasets['name']

In [90]:
# Create row index
data.index.name='orf'

In [ ]:
data.

# Print out

In [20]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [21]:
from IO.save_data_to_db2 import *

In [ ]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [22]:
save_data_to_db(data, paper_pmid)